In [1]:
import re
import math
import sys
import random

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'จบข่าว - ใบขับขี่.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 153
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 616 words
filter frequent words: 263 words
filter letter words: 260 words
filter stop words: 129 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### SDC

In [6]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(idx_corpus[i]))
#             array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def sdc(bow_corpus, min_samples, eps):
    delta_eps = eps / 10
    labels = [-1 for i in range(len(bow_corpus))]
    sims = cosine_similarity(bow_corpus)
    
    points = [i for i in range(len(bow_corpus))]
    cluster_num = 0
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            cluster_num += 1
            for p in eps_neighbors:
                labels[p] = cluster_num
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            points.remove(seed)

    while cluster_num != 0:
        cluster = [numpy.array(bow_corpus.iloc[i]) for i, label in enumerate(labels) if label == cluster_num]
        eps_temp = eps
        
        while True:
            centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
            eps_temp -= delta_eps
            
            count = 0
            for i, label in enumerate(labels):
                point = numpy.array(bow_corpus.iloc[i]).reshape(1, -1)
                if label == 0 and cosine_similarity(centroid, point) >= eps_temp:
                    cluster.append(point[0])
                    labels[i] = cluster_num
                    count += 1
            if count == 0:
                break
        
        cluster_num -= 1
    
    return labels

In [8]:
def upgrade_sdc(bow_corpus, min_samples, eps):
    delta_eps = eps / 20
    labels = [-1 for i in range(len(bow_corpus))]
    initials = [[],[]]
    
    clusters = []
    clusters.append([])
    cluster_num = 0
    
    points = [i for i in range(len(bow_corpus))]
    sims = cosine_similarity(bow_corpus)
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            cluster_num += 1
            clusters.append([])
            for p in eps_neighbors:
                clusters[cluster_num].append(numpy.array(bow_corpus.iloc[p]))
                labels[p] = cluster_num
                
                initials[0].append(p)
                if p == seed:
                    initials[1].append(p)
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            clusters[0].append((seed, numpy.array(bow_corpus.iloc[seed])))
            points.remove(seed)
    
    expandable = numpy.zeros(cluster_num + 1)    
    while numpy.sum(expandable) != -cluster_num - 1:
        eps -= delta_eps
        count = numpy.zeros(cluster_num + 1)
        for point in clusters[0]:
            if labels[point[0]] != 0:
                continue

            num = point[0]
            p = point[1].reshape(1, -1)
            max_sim = 0
            for c, cluster in enumerate(clusters[1:]):
                if expandable[c + 1] == -1:
                    continue
                centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
                if cosine_similarity(centroid, p) >= max_sim:
                    max_sim = cosine_similarity(centroid, p)
                    if max_sim >= eps:
                        labels[num] = c + 1

            if labels[num] != 0:
                count[labels[num]] += 1
                clusters[labels[num]].append(point[1])

        for i, num in enumerate(count):
            if num == 0:
                expandable[i] = -1
        
    return labels, initials

In [9]:
def generate_result(predicted_labels):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    return result

In [10]:
def eval_cluster(bow_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(bow_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(bow_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [11]:
min_samples = 7
eps = 0.3

max_compactness = 0
for i in range(10):
    # _tbow_corpus = get_bow(idx_corpus)
    _tbow_corpus = get_bow(filtered_corpus)
    _tpredicted_labels, _tinitials = upgrade_sdc(_tbow_corpus, min_samples, eps)
    _tresult = generate_result(_tpredicted_labels)
    
    compactness, _tcentroids = eval_cluster(_tbow_corpus, _tresult)
    if compactness > max_compactness:
        max_compactness = compactness
        bow_corpus = _tbow_corpus
        predicted_labels = _tpredicted_labels
        initials = _tinitials
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

75.50790617783765
(array([0, 1, 2, 3, 4, 5, 6]), array([47, 19, 16, 19, 23, 18, 11])) 



### Result

In [12]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            new_labels = [new_labels[j] if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels)

[0, 1, 2, 3, 4, 5, 6]
3 2 0.34397956112645556
[0, 1, 2, 2, 4, 5, 6]


In [13]:
class Widget:
    def __init__(self, result, initials, column_name):
        self.result = result
        self.column_name = column_name
        self.initials = initials
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new][self.column_name].iteritems():
            if index in self.initials[0]:
                if index in self.initials[1]:
                    print("@", end="")
                else:
                    print("*", end="")
            print(index, value)

In [14]:
w1 = Widget(new_result, initials, 'comment')

ToggleButtons(index=5, options=(0, 1, 2, 4, 5, 6), value=6)

*31 ใช้บัตรเดียวกับบัตรประชาชนจบ ใช้   กล้องมือถือถ่าย
*34 ใช้บัตรเดียวกับบัตรประชาชนจบ ใช้   กล้องมือถือถ่าย
*53 กฏหมายเขายึดบัตรได้หรอ แค่สามารถดูแล้วคืนว่ามีใบอนุญาติ รึป่าว
*57 ยึดไบขับขี่ไม่ได้ตอรองเล่นแง่ไม่ได้ยึดโทรสัพไม่ได้เดี๋ยวโดนลักทรัพย์
*59 คือมันก็แค่แก้แอปให้ตรยึดผ่านแอป ใครโดนยึดก็ขึ้นหน้าจอแค่นั่นเอง ยากอะไร
*88 ยึดไม่ได้ก็จ่ายน้อยหน่อย
*104 ถ้ายึดโทรศัพท์เดี๋ยวตรโดนจับเอง ถถถ
*116 ไม่มีไรให้ยึด 555
*128 ยึดมือถือเลย ไม่ชอบรึ อิอิ
*133 ดิจิตัลมันยึดไม่ได้มั้ง
@139 ยึดบัตรไม่ได้ ยึดมือถือนะนักเรียน


In [15]:
w2 = Widget(result, initials, 'comment')

ToggleButtons(index=4, options=(0, 1, 2, 3, 4, 5, 6), value=4)

*6 ตำรวจไม่รู้จะยึดใบขับขี่ยังไง ตอนเขียนใบสั่งครับ ปัญหาระดับชาติเลยสินะ
*12 ไม่ได้ยึดใบขับขี่ ก็ไม่มีคนไปจ่ายค่าปรับ
*18 มันไม่ได้เกี่ยวกับพกไม่พก มันเกี่ยวกับว่าทำใบขับขี่รึยัง ถ้าใบขับขี่นอนอยู่บ้าน มันคือทำแล้วป่าว่ะ
*20 ควยไรครับคุณตำรวจ ขนส่งเป็นผู้ออกใบขับขี่ ถ้าขนส่งบอกใช้ได้คือต้องใช้ได้ครับ คุณมีหน้าที่ปฏิบัติตามครับ ไอ้ซัซ
*21 ใครโดนใบสั่ง โดนยึดใบขับขี่ เพราะ ใช้ มาต่อทะเบียน ทำใบขับขี่ใหม่ได้เลยยยยยยย
*23 ถ้าเอาตามที่เค้าว่ากฎหมายยังไม่พร้อมก็ตามนั้นก็ดีแล้วพกใบขับขี่ใบเดียวไม่ตายห่าหรอก
*32 ไม่ใบขับขี่ ไม่เป็นไร ไม่มีใบขับขี่  ก็ไม่เป็นไร ถ้าไม้ให้ค่าลิโพ สั้ก 45 ร้อยมึงเจอยึด ตร ท ไม่ต้องกล่าวก็รู้ใว้
33 ทะเลาะกันแหละดีละ ทีนี้ขนส่งก็บอกคันไหนค้างใบสั่งก็ยังต่อภาษีได้   ต่างคนต่างหาเงินเนอะ
*38 ขนส่งแก้เผ็ดด้วยการ อนุญาตให้ประชาชนทำใบขับขี่ได้ครั้งละไม่เกิน100ใบ ใบละ10บาท ถ้าได้แบบนี้สนุกเลย เอาใบขับขี่ให้ไปกองเต็ม โรงพักให้ปลวกกินไปเลย
@40 กูว่าแล้ว ถ้ายังคงเป็นโรโบคอปยุคปัจจุบันยังไงใบขับขี่ดิจิตอลไม่มีทางเกิดแน่นอน ต่อให้มีกฎหมายออกมามันก็ไม่รับลูกหรอกเชื่อดิ เสียเวล

In [16]:
w3 = Widget(result, initials, 'tokenized_comment')

ToggleButtons(index=4, options=(0, 1, 2, 3, 4, 5, 6), value=4)

*6 ['ตำรวจ', 'ใบ', 'ใบ', 'ยึด', 'ขับขี่']
*12 ['ใบ', 'ยึด', 'ขับขี่', 'คน', 'จ่าย']
*18 ['ใบ', 'ใบ', 'ทำ', 'ขับขี่', 'ขับขี่']
*20 ['ตำรวจ', 'ใบ', 'ขับขี่', 'ขนส่ง', 'ขนส่ง']
*21 ['ใบ', 'ใบ', 'ใบ', 'ยึด', 'ขับขี่']
*23 ['ใบ', 'ใบ', 'ขับขี่', 'กฎหมาย', 'ดี']
*32 ['ใบ', 'ใบ', 'ยึด', 'ขับขี่', 'ขับขี่']
33 ['ใบ', 'ขนส่ง', 'คน', 'ดี', 'สั่ง']
*38 ['ใบ', 'ใบ', 'ใบ', 'ใบ', 'ทำ']
@40 ['ใบ', 'ขับขี่', 'กฎหมาย', 'แน่นอน', 'เวลา']
*44 ['ใบ', 'ขับขี่', 'ดี', 'ผม', 'เหมือน']
*45 ['ตำรวจ', 'ใบ', 'ใบ', 'ขับขี่', 'ขับขี่']
*47 ['ใบ', 'ใบ', 'ยึด', 'ยึด', 'ขับขี่']
*50 ['ใบ', 'ใบ', 'ใบ', 'ยึด', 'ยึด']
56 ['ใบ', 'ใบ', 'สั่ง', 'สั่ง', 'ไหม']
*69 ['ตำรวจ', 'ใบ', 'ขับขี่', 'วะ', 'ประกาศ']
70 ['ใบ', 'ขนส่ง', 'จ่าย', 'สั่ง', 'ค่า']
*71 ['ใบ', 'ยึด', 'ยึด', 'ขับขี่', 'ดี']
*80 ['ใบ', 'ใบ', 'ขับขี่', 'ขับขี่', 'ดิจิตอล']
*91 ['แน่นอน']
118 ['ใบ', 'สั่ง', 'ออนไลน์']
*140 ['ใบ', 'ยึด', 'ขับขี่']
*152 ['ตำรวจ', 'ใบ', 'ขับขี่', 'ขนส่ง', 'เรื่อง']


In [17]:
seed = 0
compare = 0

a = numpy.array(bow_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(bow_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

# print(sims[12])

[[1.]]
['บัตร', 'ข่าว', 'จบ', 'ประกัน', 'เลิก']
['บัตร', 'ข่าว', 'จบ', 'ประกัน', 'เลิก']
